# Datenvisualisierung und Auswertung

In [88]:
import sqlite3
import plotly.graph_objects as go

db = sqlite3.connect('fanfictions2.db')

def freq_graph(data, count = 30):
    x = []
    y = []
    
    for item in data:
        x.append(item[0])
        y.append(item[1])

    fig = go.Figure([go.Bar(x=x[:count], y = y[:count])])
    return fig

def get_frequency(column, data, *args):
    cursor = db.cursor()
    query = 'SELECT {} FROM {}'.format(column, data)
    cursor.execute(query)
    tags = cursor.fetchall()

    frequency = {}

    for row in tags: 
        list = row[0].split(", ")
        for item in list:
            if item != "":
               if (item in frequency):
                  frequency[item] += 1
               else:
                  frequency[item] = 1
    if not args:
        frequency = sorted(frequency.items(), key=lambda x: x[1], reverse=True)

    return frequency

def get_words(data):
    cursor = db.cursor()
    query = 'SELECT Words FROM {}'.format(data)
    cursor.execute(query)
    words = cursor.fetchall()
    words2 = []
    for w in words:
        w = str(w[0])[:-1]
        words2.append(w)
    return words2

In [37]:
co_r = get_frequency("Relationships", "co")
co_r = freq_graph(co_r, 20)
co_r

In [38]:
co_c = get_frequency("Characters", "co")
co_c_g = freq_graph(co_c, 40)
co_c_g

In [8]:
fig = go.Figure()
fig.add_trace(go.Box(y=get_words("CO"), name="Crossover", boxpoints=False))
fig.add_trace(go.Box(y=get_words("PJ"), name="Percy Jackson", boxpoints=False))
fig.add_trace(go.Box(y=get_words("HP"), name="Harry Potter", boxpoints=False))

fig.show()

### Autorenvergleich

In [96]:
ns_hp_author = get_frequency("Author", "HP", "x")
ns_co_author = get_frequency("Author", "CO", "x")
ns_pj_author = get_frequency("Author", "PJ", "x")

shared_co_hp = set(ns_hp_author).intersection(ns_co_author)
print("CO+HP:   " + str(len(shared_co_hp)))

shared_co_pj = set(ns_co_author).intersection(ns_pj_author)
print("CO+PJ:   " + str(len(shared_co_pj)))

shared_pj_hp = set(ns_hp_author).intersection(ns_pj_author)
print("PJ+HP:   " + str(len(shared_pj_hp)))

shared_pj_hp_co = set(ns_hp_author).intersection(ns_pj_author).intersection(ns_co_author)
print("CO+HP+PJ:   " + str(len(shared_pj_hp_co)))

CO+HP:   2
CO+PJ:   22
PJ+HP:   9
CO+HP+PJ:   1


In [99]:
print(len(get_frequency("Author", "CO", "x")))

373


### Additional Tag Charts

In [84]:
hp_t = get_frequency("Tags", "hp")
hp_t_g = freq_graph(hp_t, 20)
hp_t_g.update_layout(title = "HP Additional Tags" , yaxis_title="appearences", xaxis_title="tag")

hp_t_g.write_image("images/hp-addtag.png", scale = 4)

In [81]:
pj_t = get_frequency("Tags", "pj")
pj_t_g = freq_graph(pj_t, 20)
pj_t_g.update_layout(title = "PJ Additional Tags" , yaxis_title="appearences", xaxis_title="tag")

pj_t_g.write_image("images/pj-addtag.png", scale = 4)

In [82]:
co_t = get_frequency("Tags", "co")
co_t_g = freq_graph(co_t, 20)
co_t_g.update_layout(title = "CO Additional Tags" , yaxis_title="appearences", xaxis_title="tag")

co_t_g.write_image("images/co-addtag.png", scale = 4)

### CO Relationships with Colors

In [102]:
cco_r = get_frequency("Relationships", "co")

for c in cco_r:
    if c[0] == "Percy Jackson & Harry Potter":
        count1 = c[1]
        cco_r.remove(c)
    if c[0] == "Percy Jackson/Harry Potter":
        count2 = c[1]
        cco_r.remove(c)
new = ("Percy Jackson/Harry Potter", count1+count2)
cco_r.append(new)
cco_r = sorted(cco_r, key = lambda x: x[1], reverse=True)

colors = ['#1f77b4',] * 40

red = [7, 14]
for i in red:
    colors[i] = 'red'

x = []
y = []
    
for item in cco_r:
    x.append(item[0])
    y.append(item[1])

count = 30
cco_r_g = go.Figure([go.Bar(x=x[:count], y = y[:count], marker_color = colors)])
    
cco_r_g.update_layout(title = "CO Relationship Tags" , yaxis_title="appearences", xaxis_title="raltionship tag")

cco_r_g.update_yaxes(automargin=True)

cco_r_g


### CO Characters with Fandom Colors

In [66]:
cco_c = get_frequency("Characters", "co")

for c in cco_c:
    if c[0] == "Annabeth Chase (Percy Jackson)":
        count1 = c[1]
        cco_c.remove(c)
    if c[0] == "Annabeth Chase":
        count2 = c[1]
        cco_c.remove(c)
new = ("Annabeth Chase", count1+count2)
cco_c.append(new)
cco_c = sorted(cco_c, key = lambda x: x[1], reverse=True)

colors = ['#1f77b4',] * 40

red  = [0, 2, 4, 6, 8, 9, 10, 11, 12, 15, 19, 25, 27, 28]

for i in red:
    colors[i] = 'red'
    
colors[29] = 'grey'

x = []
y = []
    
for item in cco_c:
    x.append(item[0])
    y.append(item[1])

count = 30
cco_c_g = go.Figure([go.Bar(x=x[:count], y = y[:count], marker_color = colors)])
    
cco_c_g.update_layout(title = "CO Character Tags", yaxis_title="appearences", xaxis_title="character names")

cco_c_g

cco_c_g.write_image("images/co-characters-marked.png", scale = 4)

### Versuch Duplikate mit fuzzywuzzy zu erkennen (verworfen)

In [9]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import re

ns_co_tags = get_frequency("Relationships", "CO", "x")

condensed = {}

for item in ns_co_tags:
    if condensed != {}:
        for x in condensed:
                y = re.split(r'\s*[\/\&]\s*', x)
                y = "%".join(y)
                z = re.split(r'\s*[\/\&]\s*', item)
                z =  "%".join(z)
                if int(fuzz.ratio(z, y)) > 90 or int(fuzz.token_sort_ratio(z, y)) > 90 or int(fuzz.token_sort_ratio(y, z)) > 90:
                    new = 0
                    add = x
                else:
                    new = 1
    else:
        new = 1
    if new == 0:
        condensed[add] += ns_co_tags[item]
    else:
        condensed[item] = ns_co_tags[item]
                
print("ready")          

test = sorted(condensed.items(), key=lambda x: x[1], reverse=True)

testx = freq_graph(test, 20)
testx

ready
